# IndShockConsumerType_extend-Example

By William Du

The following material assumes the reader is familiar with the IndShockConsumerType documentation. See [here](https://github.com/econ-ark/HARK/blob/master/examples/ConsIndShockModel/IndShockConsumerType.ipynb) for details.

In [1]:
from __future__ import print_function
import sys 
import os
from copy import copy, deepcopy
import numpy as np
import scipy as sc
from scipy import sparse as sp
from HARK.ConsumptionSaving.ConsIndShockModel3 import IndShockConsumerType
import matplotlib.pyplot as plt


This notebook provides two examples on how to implement the following three functions:

#####  Define_Distribution_Grid 
- computes the grid of normalized market resources and the grid permanent income storing each as attributes.

##### Calc_Transition_Matrix 
- computes the transition matrix, consumption policy grid, and an asset policy grid all stored as attributes. If the problem is finite horizon, this function stores lists of transition matrices, consumption policies and asset policies grid for each period of the problem.

##### Calc_Ergodic_Dist 
- computes the ergodic distribution stored as attributes. The distribution is stored as a vector (self.VecErgDstn) and as an array with row dimension equal to the length of the grid over market resources and column dimension equal to the length of the grid over permanent income  (self.ErgDstn)


# Example 1: 
# Computing Aggregate Consumption: Transition Matrix vs Simulation

This section details how to compute aggregate consumption and aggregate assets using transition matrices and compares the results against aggregate values produced by Monte Carlo Simulate methods.

### Create and solve an Instance of IndShockConsumerType_extend 

We begin by creating an instance of the IndShockConsumerType_extend and solving his problem.

### Specify Parameters

In [2]:

Dict={
    # Parameters shared with the perfect foresight model
    "CRRA":2,                             # Coefficient of relative risk aversion
    "Rfree": 1.05**.25,                   # Interest factor on assets
    "DiscFac": 0.97,                      # Intertemporal discount factor
    "LivPrb" : [.99375],                  # Survival probability
    "PermGroFac" :[1.00],                 # Permanent income growth factor

    # Parameters that specify the income distribution over the lifecycle
   
    "PermShkStd" :  [.05],                 # Standard deviation of log permanent shocks to income
    "PermShkCount" : 5,                    # Number of points in discrete approximation to permanent income shocks
    "TranShkStd" : [.3],                   # Standard deviation of log transitory shocks to income
    "TranShkCount" : 5,                    # Number of points in discrete approximation to transitory income shocks
    "UnempPrb" : 0.05,                     # Probability of unemployment while working
    "IncUnemp" :  .2,                      # Unemployment benefits replacement rate
    "UnempPrbRet" : 0.0005,                # Probability of "unemployment" while retired
    "IncUnempRet" : 0.0,                   # "Unemployment" benefits when retired
    "T_retire" : 0,                        # Period of retirement (0 --> no retirement)
    "tax_rate" : .3,                       # Flat income tax rate (legacy parameter, will be removed in future)

    # Parameters for constructing the "assets above minimum" grid
    "aXtraMin" : 0.001,                    # Minimum end-of-period "assets above minimum" value
    "aXtraMax" : 20,                       # Maximum end-of-period "assets above minimum" value
    "aXtraCount" : 48,                     # Number of points in the base grid of "assets above minimum"
    "aXtraNestFac" : 3,                    # Exponential nesting factor when constructing "assets above minimum" grid
    "aXtraExtra" : [None],                 # Additional values to add to aXtraGrid

    # A few other parameters
    "BoroCnstArt" : 0.0,                   # Artificial borrowing constraint; imposed minimum level of end-of period assets
    "vFuncBool" : False,                   # Whether to calculate the value function during solution
    "CubicBool" : False,                   # Preference shocks currently only compatible with linear cFunc
    "T_cycle" : 1,                         # Number of periods in the cycle for this agent type

    # Parameters only used in simulation
    "AgentCount" : 10000,                  # Number of agents of this type
    "T_sim" : 2000,                        # Number of periods to simulate
    "aNrmInitMean" : np.log(1.6)-(.5**2)/2,# Mean of log initial assets
    "aNrmInitStd"  : .5,                   # Standard deviation of log initial assets
    "pLvlInitMean" : 0.0,                  # Mean of log initial permanent income
    "pLvlInitStd"  : 0.0,                  # Standard deviation of log initial permanent income
    "PermGroFacAgg" : 1.0,                 # Aggregate permanent income growth factor
    "T_age" : None,                        # Age after which simulated agents are automatically killed
    
     }



In [3]:
example1 = IndShockConsumerType(**Dict)
example1.cycles = 0
example1.solve()


GPFRaw                 = 0.990911 
GPFNrm                 = 0.993135 
GPFAggLivPrb           = 0.984718 
Thorn = APF            = 0.990911 
PermGroFacAdj          = 0.997760 
uInvEpShkuInv          = 0.997760 
VAF                    = 0.961779 
WRPF                   = 0.218203 
DiscFacGPFNrmMax       = 0.983457 
DiscFacGPFAggLivPrbMax = 0.994090 


## Method 1: Simulate 

Below we use Monte Carlo Simulation methods to compute the level of aggregate consumption and aggregate assets.

In [4]:
example1.initialize_sim()
example1.simulate() #simulate

Simulated_Consumption = np.mean((example1.state_now['mNrm'] - example1.state_now['aNrm'])*example1.state_now['pLvl']) #Compute aggregate consumption
Simulated_Assets = np.mean(example1.state_now['aNrm']*example1.state_now['pLvl']) #Compute Aggregate assets

## Method 2: Applying Transition Matrices

To compute the level of aggregate consumption using the transition matrix, we begin by defining the grid over the distribution of market resources and permanent income and store the distribution of permanent income as p . 

In [5]:
example1.Define_Distribution_Grid()
p = example1.Dist_pGrid # Grid of permanent income levels

We then calculate the transition matrix and store the consumption and asset policy grid as c and asset. The transition matrix is stored as the attribute example1.TranMatrix to be used by the Calc_ergodic_Dist function.

In [6]:
example1.Calc_Transition_Matrix()
c = example1.cPolGrid #Consumption Policy Grid
asset = example1.aPolGrid # Asset Policy Grid

Finally we compute the ergodic distribution and store the distribution as a vector. Under the hood of this function, it takes in the transition matrix calculated in the previous function (example1.TranMatrix) and computes the ergodic distribution.

In [7]:
example1.Calc_Ergodic_Dist()
vecDstn = example1.VecErgDstn # Distribution of market resources as a vector 

With the transition matrix, consumption and asset policy grid, and the distribution vector, we can now compute aggregate consumption.

In [8]:

gridc = np.zeros((len(c),len(p))) 
grida = np.zeros((len(asset),len(p)))

#Transform normalized grid of consumption and asset policy into levels
for j in range(len(p)):
    gridc[:,j] = p[j]*c # Array of all possible combinations of consumption and permanent income.
    grida[:,j] = p[j]*asset # Array of all possible combinations of assets and permanent income.
    
AggC = np.dot(gridc.flatten(),vecDstn) # Compute aggregate consumption by multiplying consumption policy grid by distribution
AggA = np.dot(grida.flatten() ,vecDstn)# Compute aggregate assets by multiplying asset policy grid by distribution

We being by defining the grid over the distribution of market resources and permanent income.

##  Comparing Outputs of Both Methods

In [9]:
print('TranMatrix Assets =' + str(AggA[0]))
print('Simulated Assets = ' +str(Simulated_Assets))

print('TranMatrix Consumption =' + str(AggC[0]))
print('Simulated Consumption = ' +str(Simulated_Consumption))

TranMatrix Assets =1.1493665253316738
Simulated Assets = 1.1878906213808793
TranMatrix Consumption =1.0067640869283523
Simulated Consumption = 1.0076918586002666


## A Different Calibration

As a robustness check, the same exercise is implemented with a different set of parameters to see whether simulated aggregates match the aggregates produced with transition matrices. 

In [10]:
example2 = IndShockConsumerType(**Dict)
example2.cycles = 0

#New Parameters
example2.IncUnemp = .5
example2.UnempPrb = .1
example2.PermShkStd = [.03]
example2.TranShkStd = [.1]
example2.tax_rate = .1
example2.Rfree = 1.02**.25
example2.DiscFac = .99

In [11]:
example2.solve()
example2.initialize_sim()
example2.simulate()


Simulated_Consumption = np.mean((example2.state_now['mNrm'] - example2.state_now['aNrm'])*example2.state_now['pLvl'])
Simulated_Assets = np.mean(example2.state_now['aNrm']*example2.state_now['pLvl'])

GPFRaw                 = 0.997453 
GPFNrm                 = 0.999692 
GPFAggLivPrb           = 0.991219 
Thorn = APF            = 0.997453 
PermGroFacAdj          = 0.997760 
uInvEpShkuInv          = 0.997760 
VAF                    = 0.981609 
WRPF                   = 0.312882 
DiscFacGPFNrmMax       = 0.990609 
DiscFacGPFAggLivPrbMax = 1.001320 


In [12]:
example2.Define_Distribution_Grid()
p = example2.Dist_pGrid # Grid of permanent income levels

example2.Calc_Transition_Matrix()
c = example2.cPolGrid #Consumption Policy Grid
asset = example2.aPolGrid # Asset Policy Grid

example2.Calc_Ergodic_Dist()
vecDstn = example2.VecErgDstn # Distribution as a vector

gridc = np.zeros((len(c),len(p)))
grida = np.zeros((len(asset),len(p)))

#Transform normalized grid of consumption and asset policy into levels
for j in range(len(p)):
    gridc[:,j] = p[j]*c
    grida[:,j] = p[j]*asset
    
AggC = np.dot(gridc.flatten(),vecDstn)
AggA = np.dot(grida.flatten() ,vecDstn)   

In [13]:
print('TranMatrix Assets =' + str(AggA[0]))
print('Simulated Assets = ' +str(Simulated_Assets))

print('TranMatrix Consumption =' + str(AggC[0]))
print('Simulated Consumption = ' +str(Simulated_Consumption))

TranMatrix Assets =1.865483816969303
Simulated Assets = 1.907460492421025
TranMatrix Consumption =0.9973327527616564
Simulated Consumption = 0.9974730909870996


Note* Accuracy decreases as standard deviation of permanent shocks increases.

# Example 2: 
# Computing the Path of Aggregate Consumption given a Pertubation in the Interest Rate

This section details an experiment where agents anticipates a change in the interest rate at period $t = 10$ and plots the paths of aggregate consumption and aggregate assets.

## Compute Steady State Aggregate Consumption

We begin by computing the steady state level of aggregate consumption. This is necesary to obtain the steady state consumption function to be inputed as the terminal solution for the next agent (see below). 

In [14]:
ss = IndShockConsumerType(**Dict) #
ss.cycles = 0
ss.solve()

ss.Define_Distribution_Grid()
p = ss.Dist_pGrid # Grid of permanent income 

ss.Calc_Transition_Matrix()
c = ss.cPolGrid # Normalized Consumption Policy grid
a = ss.aPolGrid #Normalized Asset Policy grid

ss.Calc_Ergodic_Dist()
vecDstn = ss.VecErgDstn # Distribution over market resources as a vector

gridc = np.dot( c.reshape( len(c), 1 ) , p.reshape( 1 , len(p) ) ) #Transform grid from normalized consumption to level of consumption
grida = np.dot( a.reshape( len(a), 1 ) , p.reshape( 1 , len(p) ) ) #Transform grid from normalized assets to level of assets

C_ss = np.dot(gridc.flatten(),vecDstn) # Compute steady state aggregate consumption
A_ss = np.dot(grida.flatten() ,vecDstn)# Compute steady state aggregate assets     



GPFRaw                 = 0.990911 
GPFNrm                 = 0.993135 
GPFAggLivPrb           = 0.984718 
Thorn = APF            = 0.990911 
PermGroFacAdj          = 0.997760 
uInvEpShkuInv          = 0.997760 
VAF                    = 0.961779 
WRPF                   = 0.218203 
DiscFacGPFNrmMax       = 0.983457 
DiscFacGPFAggLivPrbMax = 0.994090 


In [15]:
print('steady state Consumption: ' + str(C_ss[0]))
print('steady state Assets: ' + str(A_ss[0]))

steady state Consumption: 1.0067640866750511
steady state Assets: 1.1493665250639191


## Solving an Agent who Anticipates a Change in the Interest Rate

We need to specify an agent who faces a finite horizon problem. The agent now takes in lists of parameters. Each list is the length of the horizon ($T=30$) in the model. Elements of each list indicate the value of the parameter faced in the respective period.

In [16]:
params = deepcopy(Dict)
params['T_cycle']= 30 # Set Horizon Length T = 30
params['LivPrb']= params['T_cycle']*[ss.LivPrb[0]] # list of living probabilities
params['PermGroFac']=params['T_cycle']*[1] # list of permanent growth factor
params['PermShkStd'] = params['T_cycle']*[ss.PermShkStd[0]] # list of permanent shock standard deviations
params['TranShkStd']= params['T_cycle']*[ss.TranShkStd[0]] # list of transitory shock standard deviations
params['Rfree'] = params['T_cycle']*[ss.Rfree] # list of interest rates (will be changed shortly)

FinHorizonAgent = IndShockConsumerType(**params) # input these parameters into the agent.
FinHorizonAgent.cycles = 1
FinHorizonAgent.del_from_time_inv('Rfree') #delete Rfree from time invariant list since it varies overtime
FinHorizonAgent.add_to_time_vary('Rfree')
FinHorizonAgent.IncShkDstn = params['T_cycle']*ss.IncShkDstn

#Below is crucial!!
FinHorizonAgent.cFunc_terminal_ = deepcopy(ss.solution[0].cFunc) # Set Terminal Solution as Steady State Consumption Function



### Implement perturbation in interest rate

We specify the list of interest rates the agent faces by FinHorizonAgent.Rfree.
Note the 10th element of the list FinHorizonAgent.Rfree has an interest rate that is $.01$ greater than the steady state level.

In [17]:
dx=.01 # Change in the Interest Rate
i = 10 # Period in which the change in the interest rate occurs
FinHorizonAgent.Rfree = (i)*[ss.Rfree] + [ss.Rfree + dx] + (params['T_cycle'] - i -1 )*[ss.Rfree] # Sequence of interest rates the agent faces


### Solve Agent and Calculate Transition Matrices

In [18]:
FinHorizonAgent.solve() #Solve the Agent's problem
FinHorizonAgent.Define_Distribution_Grid() 
FinHorizonAgent.Calc_Transition_Matrix()


Since this is a finite horizon problem now, example.Calc_Transition_Matrix() now generates:
- a list of transition matrices for each period (example.TranMatrix)
- a list of consumption policies for each period (example.cPolGrid)
- a list of asset policies for each period (example.aPolGrid)



In [19]:
AggC_List =[] # List of aggregate consumption for each period t 
AggA_List =[] # List of aggregate assets for each period t 

dstn = vecDstn # Initial distribution set as steady state distribution

T=params['T_cycle']
for i in range(T):

    p = FinHorizonAgent.Dist_pGrid[i]# Permanent income Grid this period
    c = FinHorizonAgent.cPolGrid[i] # Consumption Policy Grid this period
    a = FinHorizonAgent.aPolGrid[i] # Asset Policy Grid this period
        
    gridc = np.dot( c.reshape( len(c), 1 ) , p.reshape( 1 , len(p) ) ) #Transform grid from normalized consumption to level of consumption
    C = np.dot( gridc.flatten() , dstn )  # Compute Aggregate Consumption this period
    AggC_List.append(C)
    
    grida = np.dot( a.reshape( len(a), 1 ) , p.reshape( 1 , len(p) ) ) #Transform grid from normalized assets to level of assets
    A = np.dot( grida.flatten() , dstn ) # Compute Aggregate Assets this period
    AggA_List.append(A)
    
    dstn = np.dot(FinHorizonAgent.TranMatrix[i],dstn) # Iterate Distribution forward
    
#Transform Lists into tractable arrays
AggC_List  = np.array(AggC_List).T[0] 
AggA_List  = np.array(AggA_List).T[0]

### Path of Aggregate Consumption given an anticipated interest rate shock at $t=10$

In [ ]:
plt.plot(AggC_List,label= 'Perturbed Consumption' ) # Plots Aggregate Consumption given a change in there interest rate during period 10
plt.plot(np.ones( len(AggC_List) )* C_ss,label = 's.s. Consumption', color = 'k') #Plots steady state Aggregate Consumption
plt.ylim([1.004,1.01])
plt.legend()
plt.show()
    

### Path of Aggregate Assets given an anticipated interest rate shock at $t=10$

In [ ]:
plt.plot(AggA_List, label = 'Perturbed Assets') # Plots Aggregate Assets given a change in there interest rate during period 10
plt.plot(np.ones( len(AggA_List) )* A_ss,label= 's.s. assets', color= 'k') #Plots steady state Aggregate Assets
plt.legend()
plt.show()